<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-данных" data-toc-modified-id="Загрузка-библиотек-и-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка библиотек и данных</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка библиотек и данных

In [23]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [24]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from catboost import CatBoostClassifier
from pymystem3 import Mystem 
import warnings
warnings.filterwarnings('ignore')

from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [26]:
comments.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [27]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [28]:
print('Дубликатов:', comments.duplicated().sum())

Дубликатов: 0


In [29]:
print('Пропусков:')
comments.isna().sum()

Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [30]:
print('Соотношение в целевом признаке:') 
comments.toxic.value_counts(normalize=True)

Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

Выводы:
Всего у нас 159292 комментария. Пропуски и дубликаты отсутствуют. Комментарии на английском языке, могут содержаться неизвестные структуры данных (такие как "\nMore\n"). Стоит избавиться от колонки аннэймд, так как она дублирует индексы.

### Предобработка

In [31]:
comments = comments.drop(['Unnamed: 0'], axis=1)

In [32]:
def words_only(text):
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text.lower()).split())

def lemmatize_text(text, nlp):
    return [token.lemma_ for token in nlp(text)]

def remove_stopwords(text, stopwords):
    return [word for word in text if not word in stopwords and len(word) > 1]

def clean_text(text, nlp, stopwords):

    # очистка от символов
    text_words = words_only(text)

    # лемматизация
    text_lemmas = lemmatize_text(text_words, nlp)

    # очистка от стоп-слов
    return ' '.join(remove_stopwords(text_lemmas, stopwords))

In [33]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

nltk.download('stopwords')
stopwords = nltk_stopwords.words('english')

lemmas = []

for i in range(len(comments)):

    lemmas.append(clean_text(comments['text'][i], nlp, stopwords))

comments['text'] = lemmas

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
comments.head()

,text,toxic
0,explanation edit make username hardcore metall...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really try edit war guy constantly rem...,0
3,make real suggestion improvement wonder sectio...,0
4,sir hero chance remember page,0


In [35]:
#разделяем выборки в соотношении 80/10/10:
features = comments.drop(['toxic'], axis=1) 
target = comments.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [36]:
#размер выборок
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [37]:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений "1" в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений "1" в тренировочной выборке: 0.10166911239631807


In [38]:
#уменьшаю количество "0" в выборках train:

comments_train = comments.iloc[target_train.index]
target_train_0 = comments_train[comments_train['toxic'] == 0]['toxic']
target_train_1 = comments_train[comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1 к 0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

Соотношение 1 к 0 в тренировочной выборке:
0    0.5
1    0.5
Name: toxic, dtype: float64

(25912,)
(25912,)


В рамках предобработки мы удалили ненужный столбец, лемматезировали текст, избавили его от ненужных для анализа символов и стоп-слов. Также мы сбалансировали веса в целевом признаке.

## Обучение

### Логистическая регрессия

In [39]:
features_train = features_train.text

In [40]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 
                                                  
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('F1 логистической регрессии на тесте =', round(lr_test_f1,2))

F1 логистической регрессии = 0.77
при параметрах {'lr__C': 5, 'lr__class_weight': 'balanced', 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'saga'}

F1 логистической регрессии на валидации = 0.77

F1 логистической регрессии на тесте = 0.77
CPU times: user 9min 24s, sys: 3min 52s, total: 13min 16s
Wall time: 13min 17s


### Дерево решений

In [46]:
%%time

#обучение:
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')), 
                    ('random_over_sampler', RandomOverSampler()),
                    ('DecisionTreeClassifier', DecisionTreeClassifier())])
    
parameters = {'DecisionTreeClassifier__max_depth': [x for x in range(50,100,2)]}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
dtc_test_f1 = f1_score(target_test, predictions_test)
print('F1 дерева решений на тесте =', round(dtc_test_f1,2))

F1 дерева решений = 0.82
при параметрах {'DecisionTreeClassifier__max_depth': 96}

F1 дерева решений на валидации = 0.59

F1 дерева решений на тесте = 0.59
CPU times: user 5min 48s, sys: 657 ms, total: 5min 49s
Wall time: 5min 49s


### Случайный лес

In [47]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([12345]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('F1 случайного леса =', round(rfc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации =', round(rfc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
rfc_test_f1 = f1_score(target_test, predictions_test)
print('F1 случайного леса на тесте =', round(rfc_test_f1,2))

F1 случайного леса = 0.78
при параметрах {'rfc__class_weight': 'balanced', 'rfc__criterion': 'entropy', 'rfc__max_depth': 9, 'rfc__n_estimators': 29, 'rfc__random_state': 12345}

F1 случайного леса на валидации = 0.36

F1 случайного леса на тесте = 0.36
CPU times: user 7min 48s, sys: 1.11 s, total: 7min 49s
Wall time: 7min 49s


### CatBoostClassifier

In [48]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} #вот вообще не уверена, что class_weights тут нужен

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()

#тестирование:
predictions_test = gscv.predict(features_test.text)
cbc_test_f1 = f1_score(target_test, predictions_test)
print('F1 CatBoostClassifier на тесте =', round(cbc_test_f1,2))

F1 CatBoostClassifier = 0.88
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации = 0.73

F1 CatBoostClassifier на тесте = 0.73
CPU times: user 9min 11s, sys: 3.59 s, total: 9min 14s
Wall time: 9min 21s


## Выводы

In [49]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        rfc_valid_f1],
        
        'F1 на тестовой выборке (финальный)': [lr_test_f1,
                                               dtc_test_f1,
                                               cbc_test_f1,
                                               rfc_test_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на тестовой выборке (финальный)', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке,F1 на тестовой выборке (финальный)
LogisticRegression,0.766619,0.766770,0.770775
CatBoostClassifier,0.877837,0.731931,0.728441
DecisionTreeClassifier,0.822708,0.591025,0.592953
RandomForestClassifier,0.783349,0.363170,0.364452


**Выводы:**
  
В рамках предобработки мы загрузили данные, проверили их на наличие пропусков и дубликатов. Избавились от ненужной колонки, которая дублировала индекс, лемматизировали текст, а также избавились от ненужных символов и стоп-слов. Сбалансировали данные и обучили 4 модели. Лучший показатель показала логистическая регрессия с F1 = 0.77, что означает, что нам подходит только данная модель.